In [1]:
include("../main.jl")
using PyCall;
using Revise;
using Gen;
using CairoMakie
using Serialization
sepsis_gym = pyimport("custom_sepsis");

In [2]:
# models = [MCMCModel(type, choicemap(), [], []) for type in [:SimplePPL, :Softmax]]
# histories = [PPHistoryRun("$(label_dict[model.type])-finer", model) for model in models]

In [1]:
# # initialize, get mcmc running --> needs 200 iterations
# for history in histories
#     update_model!(history.model, 1)
#     run_history_mcmc!(history, 1, 200, 3)
# end

In [2]:
# Parameters
start_exponent = 0  # Start at 10^0 = 1
stop_exponent = 3   # End at 10^3 = 1000
num_points = 15     # Number of points in the range

# Calculate the step size
step = (stop_exponent - start_exponent) / (num_points)

# Loop through the exponents and compute values
values = [1]
for i in 1:num_points
    exponent = start_exponent + i * step
    value = Int(round(10^exponent))
    push!(values, value)
end
indices = sort(collect(Set(values)))


15-element Vector{Any}:
    2
    3
    4
    6
   10
   16
   25
   40
   63
  100
  158
  251
  398
  631
 1000

In [ ]:

histories = [
    [] for _ in 1:5
]
policies = [
    [] for _ in 1:5
]
for history_size in indices
    for sample in 1:5
        pols = [sepsis_gym.random_policy() for _ in 1:history_size]
        episodes = [sepsis_gym.run_episode(pols[i]) for i in 1:history_size]
        push!(histories[sample], episodes)
        push!(policies[sample], pols)
        for type in [:SimplePPL, :Softmax]
            model = MCMCModel(type, choicemap(), [], []) 
            history = PPHistoryRun("$(label_dict[model.type])-$(history_size)-$sample", model)
            update_model_history!(history.model, episodes, [to_policy(p) for p in pols])
            run_history_mcmc!(history, history_size, 200+history_size, 1)
            save_history_jld(history)
            println("Finished episode ", history.model.type, " ", history_size)
        end
    end
end

Finished episode SimplePPL 16
Finished episode Softmax 16
Finished episode SimplePPL 16
Finished episode Softmax 16
Finished episode SimplePPL 25
Finished episode Softmax 25
Finished episode SimplePPL 25
Finished episode Softmax 25
Finished episode SimplePPL 25
Finished episode Softmax 25
Finished episode SimplePPL 25
Finished episode Softmax 25
Finished episode SimplePPL 25
Finished episode Softmax 25
Finished episode SimplePPL 40
Finished episode Softmax 40
Finished episode SimplePPL 40
Finished episode Softmax 40
Finished episode SimplePPL 40
Finished episode Softmax 40
Finished episode SimplePPL 40
Finished episode Softmax 40
Finished episode SimplePPL 40
Finished episode Softmax 40
Finished episode SimplePPL 63
Finished episode Softmax 63
Finished episode SimplePPL 63
Finished episode Softmax 63
Finished episode SimplePPL 63
Finished episode Softmax 63
Finished episode SimplePPL 63
Finished episode Softmax 63
Finished episode SimplePPL 63
Finished episode Softmax 63
Finished episo

In [8]:
other_histories = [load_jld("../../../data/mcmc/history/SimplePPL.jld"), load_jld("../../../data/mcmc/history/SoftmaxPPL.jld")]

2-element Vector{PPHistoryRun}:
 PPHistoryRun("SimplePPL", MCMCModel(:SimplePPL, DynamicChoiceMap(Dict{Any, Any}(), Dict{Any, Any}(:episodes => DynamicChoiceMap(Dict{Any, Any}(), Dict{Any, Any}(1144 => DynamicChoiceMap(Dict{Any, Any}(), Dict{Any, Any}(:trajectory => DynamicChoiceMap(Dict{Any, Any}(), Dict{Any, Any}(5 => DynamicChoiceMap(Dict{Any, Any}(:o2 => LOW, :hr => LOW, :bp => HIGH, :glu => NORMAL), Dict{Any, Any}()), 4 => DynamicChoiceMap(Dict{Any, Any}(:o2 => NORMAL, :hr => LOW, :bp => HIGH, :glu => NORMAL), Dict{Any, Any}()), 2 => DynamicChoiceMap(Dict{Any, Any}(:o2 => NORMAL, :hr => LOW, :bp => HIGH, :glu => NORMAL), Dict{Any, Any}()), 3 => DynamicChoiceMap(Dict{Any, Any}(:o2 => NORMAL, :hr => LOW, :bp => HIGH, :glu => NORMAL), Dict{Any, Any}()), 1 => DynamicChoiceMap(Dict{Any, Any}(:o2 => NORMAL, :hr => LOW, :bp => NORMAL, :glu => NORMAL), Dict{Any, Any}()))))), 1175 => DynamicChoiceMap(Dict{Any, Any}(), Dict{Any, Any}(:trajectory => DynamicChoiceMap(Dict{Any, Any}(), Dict{An

In [9]:
struct HistoryType
    # policies::Dict{Int, Vector{Policy}}
    mean_rewards::Dict{Int, Vector{Float64}}
    name::String
    info::Dict
end
using JSON3

function save_ts_to_json(ts_instance::HistoryType, file_path::String)
    json_data = Dict(
        "mean_rewards" => ts_instance.mean_rewards,
        "name" => ts_instance.name,
        "info" => ts_instance.info
    )
    JSON3.write(file_path, json_data)
end

save_ts_to_json (generic function with 1 method)

In [7]:
for (i, history) in enumerate(histories)
    other_history = other_histories[i]
    for (key, rewards) in other_history.mean_rewards
        if haskey(history.mean_rewards, key)
            append!(history.mean_rewards[key], rewards)
        else
            history.mean_rewards[key] = rewards
        end
    end
    save_ts_to_json(HistoryType(history.mean_rewards, "$(label_dict[history.model.type])-merged", Dict()), "../../../data/mcmc/ts/$(label_dict[history.model.type])-merged.json")
end

UndefVarError: UndefVarError: `other_histories` not defined